In [16]:
import numpy as np
import pandas as pd
from ase.io import read
import matplotlib.pyplot as plt
from funciones import get_molecules
from funciones import get_data

# Reading the original file, with all the frames of relaxation

In [17]:
frames = read("pw.out", index=":")
len(frames)

200

In [18]:
frames[0][0].symbol

'Cl'

# Getting the indeces of the molecules in the file.

The indeces never change, so it is only needed to get the indeces of one frame

In [19]:
sol = get_molecules(["alcl3_opt.xyz","1-Ethyl-3-methylimidazolium_chloride_opt.xyz"], "pw.out")
len(sol) # number of molecules in the file "pw1.out"

16

# Getting the center of masses of every molecule in every frame

In [20]:
frames_pos_data = np.empty(len(frames), dtype=list)
for i in range(len(frames)): # loop for frames in the quantum espresso file
	center_of_masses = []
	for j in range(len(sol)):
		center_of_masses.append(frames[i].get_center_of_mass(scaled=False,indices= sol[j].idx_list)) # Collecting the center of masses of every molecule in the i frame

	frames_pos_data[i] = center_of_masses # Once it's finished the loop for molecules, the variable "center_of_masses" has the center of mass of every molecule in the i frame, so 
										# it will be storaged inside the list "frames_pos_data" in the corresponding index

## Example: How to get the center of masses of the molecules in frame cero

In [21]:
frames_pos_data[0]

[array([2.35676109, 9.51703369, 5.91976337]),
 array([11.66973826, 11.73447346,  2.61054945]),
 array([13.09145252, 10.342371  ,  2.80793469]),
 array([ 6.86188744, 12.59468001,  6.50676544]),
 array([13.04127704, 13.71216404,  1.95107938]),
 array([16.22330128,  3.24509168,  3.81353755]),
 array([5.73301073, 7.19385159, 3.31943936]),
 array([4.35668667, 3.47223094, 2.06974058]),
 array([1.55820976, 5.38013388, 3.47770691]),
 array([ 8.26376811, 12.82418606,  3.94332193]),
 array([10.46158171, 16.17660857,  3.31831114]),
 array([16.14791557, 14.35439918,  3.12867492]),
 array([ 4.75509718, 11.32482593,  2.72251137]),
 array([7.46425374, 4.8835632 , 5.18880222]),
 array([12.37109264,  3.0764546 ,  3.28383592]),
 array([15.92623456,  7.27868358,  4.05184713])]

# The center of mass of the mixture in the final frame

In [22]:
all_center_of_mass = frames[-1].get_center_of_mass(scaled=False)
all_center_of_mass

array([9.37359906, 9.17095745, 3.62324235])

## Moving the reference from the cell axis to the axis in the center of mass of the final configuration of the mixture

In [23]:
frames_pos_data_T = np.empty(len(frames), dtype=list)
for i in range(len(frames)):
	frames_pos_data_T[i] = frames_pos_data[i] - all_center_of_mass # Now the origin is moved to the center of mass of the mixture in the final configuration.

In [24]:
len(frames_pos_data_T)

200

# Calculating the radial distance from the center of mass of the mixture to the each molecule

In [25]:
frames_radius_distance = np.empty(len(frames), dtype=list)
for i in range(len(frames)): # loop for every frame
	aux = np.empty(len(sol),dtype=float)
	for j in range(len(sol)): # loop for every molecule
		aux[j] = np.sqrt(frames_pos_data_T[i][j].dot(frames_pos_data_T[i][j])) # Collecting the radius distance of molecule j in frame i
	
	frames_radius_distance[i] = aux # Collecting the radius distance of all the molecules in frame i

In [26]:
frames_radius_distance = np.vstack(frames_radius_distance, dtype=float)
frames_radius_distance

array([[7.39119697, 3.58739689, 3.98238253, ..., 4.94755765, 6.8002301 ,
        6.83384478],
       [7.39385445, 3.58607577, 3.98358967, ..., 4.94875325, 6.7991774 ,
        6.83350565],
       [7.39794649, 3.58445354, 3.98503122, ..., 4.95025678, 6.79803896,
        6.83343692],
       ...,
       [8.39889705, 3.68527099, 4.50675737, ..., 5.12271079, 6.54306225,
        7.12455193],
       [8.40029204, 3.68994087, 4.50960158, ..., 5.12406025, 6.54290726,
        7.12530313],
       [8.40167842, 3.69468123, 4.5124352 , ..., 5.12541644, 6.54275252,
        7.12605282]], shape=(200, 16))

In [27]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Patch 

# Crear figura
fig, ax = plt.subplots(figsize=(12, 8))

labels = [f"Molecule {i+1}" for i in range(len(frames_radius_distance[0]))]

color_list = ['slategrey','forestgreen']

# Crear lista de colores personalizada
colors = [f"{color_list[0]}" if i < 7 else f"{color_list[1]}" for i in range(len(labels))]

x = np.arange(len(labels))
bars = ax.bar(x, frames_radius_distance[0], tick_label=labels, color = colors)

plt.setp(ax.get_xticklabels(), rotation=90)
ax.set_ylim(0, np.max(frames_radius_distance) * 1.1)
ax.set_ylabel("Radial Distance from the final CM of the mixture to each molecule")

# Leyenda manual
legend_elements = [
    Patch(facecolor=f"{color_list[0]}", label=r'$AlCl_3$'),
    Patch(facecolor=f"{color_list[1]}", label='1-Ethyl-3-methylimidazolium_chloride')
]
ax.legend(handles=legend_elements, loc='upper right')

# Función de actualización
def update(frame):
    y_vals = frames_radius_distance[frame]
    for bar, y in zip(bars, y_vals):
        bar.set_height(y)
    ax.set_title(f"Frame {frame}", pad =10)
    return bars

plt.tight_layout()
plt.subplots_adjust(top=0.88)
# Crear animación
ani = animation.FuncAnimation(fig, update, frames=frames_radius_distance.shape[0], interval=50, blit=False)

# Guardar el video
Writer = animation.writers['ffmpeg']
writer = Writer(fps=10, metadata=dict(artist='Me'), bitrate=1800)
ani.save("animacion.mp4", writer=writer)

plt.close(fig)  # Importante si no querés mostrar la figura estática

#HTML(anim.to_jshtml())

In [29]:
aux = get_data(["alcl3_opt.xyz","1-Ethyl-3-methylimidazolium_chloride_opt.xyz"], "pw.out")


cols = aux.columns

# Iteramos en pasos de 3 (x,y,z)
for i in range(0, len(cols), 3):
    # Columnas de x,y,z para esa molécula
    x_col = cols[i]
    y_col = cols[i+1]
    z_col = cols[i+2]

    # Sumamos el desplazamiento
    aux[x_col] = aux[x_col] - all_center_of_mass[0]
    aux[y_col] = aux[y_col] - all_center_of_mass[1]
    aux[z_col] = aux[z_col] - all_center_of_mass[2]

aux

,Cl_1_x,Cl_1_y,Cl_1_z,Cl_2_x,Cl_2_y,Cl_2_z,Cl_3_x,Cl_3_y,Cl_3_z,Al_4_x,...,NNCCCCCCHHHHHHHHHHHCl_13_z,NNCCCCCCHHHHHHHHHHHCl_14_x,NNCCCCCCHHHHHHHHHHHCl_14_y,NNCCCCCCHHHHHHHHHHHCl_14_z,NNCCCCCCHHHHHHHHHHHCl_15_x,NNCCCCCCHHHHHHHHHHHCl_15_y,NNCCCCCCHHHHHHHHHHHCl_15_z,NNCCCCCCHHHHHHHHHHHCl_16_x,NNCCCCCCHHHHHHHHHHHCl_16_y,NNCCCCCCHHHHHHHHHHHCl_16_z
0,-5.402088,-0.790897,1.302605,-6.759713,2.503598,2.705154,-8.888713,-0.674472,2.881803,-7.016837,...,-0.900731,-1.909345,-4.287394,1.565560,2.997494,-6.094503,-0.339406,6.552635,-1.892274,0.428605
1,-5.410077,-0.786917,1.306762,-6.760303,2.505752,2.710357,-8.885480,-0.675149,2.882060,-7.018233,...,-0.901393,-1.909414,-4.288319,1.566722,2.999587,-6.092227,-0.340672,6.552387,-1.892279,0.426971
2,-5.420585,-0.783382,1.312860,-6.762082,2.506884,2.716797,-8.882516,-0.674565,2.883344,-7.019976,...,-0.902399,-1.909296,-4.289440,1.568544,3.001583,-6.089903,-0.341928,6.552424,-1.892190,0.425687
3,-5.433551,-0.780305,1.320871,-6.765047,2.506992,2.724448,-8.879800,-0.672744,2.885637,-7.022078,...,-0.903743,-1.908994,-4.290750,1.571021,3.003485,-6.087523,-0.343177,6.552745,-1.892015,0.424761
4,-5.448547,-0.777772,1.330534,-6.769115,2.506080,2.733053,-8.877381,-0.669769,2.888845,-7.024489,...,-0.905378,-1.908519,-4.292198,1.574074,3.005240,-6.085148,-0.344388,6.553335,-1.891767,0.424222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-6.198692,-0.484260,3.034261,-7.263676,3.011688,3.316505,-9.561825,0.317156,3.457254,-7.611010,...,-0.853284,-1.665067,-4.387545,2.047378,2.816195,-5.906324,0.011568,6.701955,-2.288794,0.763528
196,-6.198891,-0.480263,3.045188,-7.264036,3.015056,3.310911,-9.561377,0.322578,3.456834,-7.612029,...,-0.849884,-1.665213,-4.387996,2.049627,2.815989,-5.906233,0.017791,6.701493,-2.291998,0.765018
197,-6.199131,-0.476247,3.056043,-7.264369,3.018434,3.305320,-9.560942,0.327945,3.456452,-7.613006,...,-0.846517,-1.665367,-4.388443,2.051897,2.815808,-5.906125,0.024025,6.701031,-2.295192,0.766502
198,-6.199412,-0.472214,3.066822,-7.264673,3.021828,3.299735,-9.560529,0.333252,3.456108,-7.613937,...,-0.843188,-1.665528,-4.388886,2.054189,2.815652,-5.905999,0.030269,6.700569,-2.298376,0.767978


In [30]:
df = aux.iloc[:,-16*3:]
df

,ClClClAl_1_x,ClClClAl_1_y,ClClClAl_1_z,ClClClAl_2_x,ClClClAl_2_y,ClClClAl_2_z,ClClClAl_3_x,ClClClAl_3_y,ClClClAl_3_z,ClClClAl_4_x,...,NNCCCCCCHHHHHHHHHHHCl_13_z,NNCCCCCCHHHHHHHHHHHCl_14_x,NNCCCCCCHHHHHHHHHHHCl_14_y,NNCCCCCCHHHHHHHHHHHCl_14_z,NNCCCCCCHHHHHHHHHHHCl_15_x,NNCCCCCCHHHHHHHHHHHCl_15_y,NNCCCCCCHHHHHHHHHHHCl_15_z,NNCCCCCCHHHHHHHHHHHCl_16_x,NNCCCCCCHHHHHHHHHHHCl_16_y,NNCCCCCCHHHHHHHHHHHCl_16_z
0,-7.016838,0.346076,2.296521,2.296139,2.563516,-1.012693,3.717853,1.171414,-0.815308,-2.511712,...,-0.900731,-1.909345,-4.287394,1.565560,2.997494,-6.094503,-0.339406,6.552635,-1.892274,0.428605
1,-7.018542,0.347927,2.299587,2.294493,2.562910,-1.013279,3.720011,1.171060,-0.811865,-2.510953,...,-0.901393,-1.909414,-4.288319,1.566722,2.999587,-6.092227,-0.340672,6.552387,-1.892279,0.426971
2,-7.021373,0.349704,2.303832,2.291760,2.562338,-1.015170,3.723073,1.168915,-0.807983,-2.510137,...,-0.902399,-1.909296,-4.289440,1.568544,3.001583,-6.089903,-0.341928,6.552424,-1.892190,0.425687
3,-7.025312,0.351403,2.309237,2.287957,2.561794,-1.018341,3.727024,1.165008,-0.803671,-2.509254,...,-0.903743,-1.908994,-4.290750,1.571021,3.003485,-6.087523,-0.343177,6.552745,-1.892015,0.424761
4,-7.030226,0.352972,2.315626,2.283216,2.561293,-1.022667,3.731729,1.159480,-0.799063,-2.508315,...,-0.905378,-1.908519,-4.292198,1.574074,3.005240,-6.085148,-0.344388,6.553335,-1.891767,0.424222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-7.661836,0.957174,3.297608,2.240538,2.052524,-2.069091,4.367401,0.447104,-0.992587,-2.765621,...,-0.853284,-1.665067,-4.387545,2.047378,2.816195,-5.906324,0.011568,6.701955,-2.288794,0.763528
196,-7.662072,0.961639,3.299362,2.241334,2.053577,-2.075230,4.369581,0.449934,-0.994713,-2.768720,...,-0.849884,-1.665213,-4.387996,2.049627,2.815989,-5.906233,0.017791,6.701493,-2.291998,0.765018
197,-7.662307,0.966092,3.301090,2.242129,2.054726,-2.081390,4.371760,0.452752,-0.996790,-2.771843,...,-0.846517,-1.665367,-4.388443,2.051897,2.815808,-5.906125,0.024025,6.701031,-2.295192,0.766502
198,-7.662541,0.970530,3.302794,2.242920,2.055971,-2.087572,4.373939,0.455557,-0.998816,-2.774989,...,-0.843188,-1.665528,-4.388886,2.054189,2.815652,-5.905999,0.030269,6.700569,-2.298376,0.767978


In [31]:
# Obtener nombres únicos de las moléculas sin el sufijo _x, _y, _z
base_names = sorted(set(col.rsplit('_', 1)[0] for col in df.columns if col.endswith('_x')))

# Para cada molécula, calcular su distancia radial y agregarla al DataFrame
for name in base_names:
    x = df[f"{name}_x"]
    y = df[f"{name}_y"]
    z = df[f"{name}_z"]
    df[f"{name}_r"] = np.sqrt(x**2 + y**2 + z**2)

In [32]:
DF = df.iloc[:,-16:]

DF

,ClClClAl_1_r,ClClClAl_2_r,ClClClAl_3_r,ClClClAl_4_r,ClClClAl_5_r,ClClClAl_6_r,ClClClAl_7_r,NNCCCCCCHHHHHHHHHHHCl_10_r,NNCCCCCCHHHHHHHHHHHCl_11_r,NNCCCCCCHHHHHHHHHHHCl_12_r,NNCCCCCCHHHHHHHHHHHCl_13_r,NNCCCCCCHHHHHHHHHHHCl_14_r,NNCCCCCCHHHHHHHHHHHCl_15_r,NNCCCCCCHHHHHHHHHHHCl_16_r,NNCCCCCCHHHHHHHHHHHCl_8_r,NNCCCCCCHHHHHHHHHHHCl_9_r
0,7.391197,3.587397,3.982383,5.132765,6.072112,9.059278,4.153929,3.831482,7.096185,8.544240,5.175039,4.947558,6.800230,6.833845,7.749727,8.687453
1,7.393854,3.586076,3.983590,5.131806,6.073314,9.057940,4.155905,3.830888,7.097068,8.543688,5.173546,4.948753,6.799177,6.833506,7.749079,8.689967
2,7.397946,3.584454,3.985031,5.131212,6.075111,9.056821,4.158320,3.831163,7.098418,8.543705,5.172270,4.950257,6.798039,6.833437,7.748801,8.692563
3,7.403450,3.582534,3.986708,5.130980,6.077502,9.055915,4.161174,3.832287,7.100229,8.544289,5.171224,4.952060,6.796810,6.833638,7.748885,8.695239
4,7.410181,3.580383,3.988571,5.131108,6.080418,9.055239,4.164385,3.834197,7.102447,8.545419,5.170451,4.954101,6.795520,6.834102,7.749319,8.697921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,8.396078,3.676140,4.501035,5.548792,6.877915,9.864637,4.514498,4.512585,8.198022,9.283008,6.040220,5.120035,6.543375,7.123044,8.403738,9.075169
196,8.397493,3.680671,4.503902,5.552458,6.880120,9.869884,4.512933,4.515222,8.198902,9.282401,6.040580,5.121369,6.543218,7.123799,8.406001,9.077428
197,8.398897,3.685271,4.506757,5.556162,6.882268,9.875130,4.511353,4.517794,8.199762,9.281769,6.040878,5.122711,6.543062,7.124552,8.408229,9.079651
198,8.400292,3.689941,4.509602,5.559904,6.884358,9.880376,4.509758,4.520298,8.200604,9.281114,6.041120,5.124060,6.542907,7.125303,8.410424,9.081835


In [33]:
DF.describe()

,ClClClAl_1_r,ClClClAl_2_r,ClClClAl_3_r,ClClClAl_4_r,ClClClAl_5_r,ClClClAl_6_r,ClClClAl_7_r,NNCCCCCCHHHHHHHHHHHCl_10_r,NNCCCCCCHHHHHHHHHHHCl_11_r,NNCCCCCCHHHHHHHHHHHCl_12_r,NNCCCCCCHHHHHHHHHHHCl_13_r,NNCCCCCCHHHHHHHHHHHCl_14_r,NNCCCCCCHHHHHHHHHHHCl_15_r,NNCCCCCCHHHHHHHHHHHCl_16_r,NNCCCCCCHHHHHHHHHHHCl_8_r,NNCCCCCCHHHHHHHHHHHCl_9_r
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,8.044677,3.536958,4.207782,5.327446,6.507183,9.394619,4.417263,4.157257,7.693822,9.005478,5.608579,5.044126,6.656505,7.010841,8.051366,8.862276
std,0.308250,0.051765,0.163786,0.124121,0.250878,0.263041,0.128115,0.211444,0.385001,0.262172,0.309191,0.042777,0.088971,0.093054,0.214356,0.114632
min,7.391197,3.480146,3.982383,5.130980,6.072112,9.054633,4.153929,3.830888,7.096185,8.543688,5.169862,4.947558,6.542753,6.833437,7.748801,8.687453
25%,7.811524,3.496208,4.058550,5.214487,6.295143,9.146657,4.307101,3.976332,7.319869,8.766795,5.305914,5.018249,6.563030,6.935386,7.853965,8.759943
50%,8.140969,3.523047,4.175921,5.337113,6.511654,9.359568,4.463262,4.126189,7.726467,9.076980,5.606238,5.044444,6.653449,7.040074,8.030251,8.846836
75%,8.313246,3.563059,4.353399,5.423118,6.740137,9.624052,4.533886,4.350962,8.078933,9.262769,5.917487,5.074413,6.741544,7.090453,8.246751,8.955187
max,8.401678,3.694681,4.512435,5.563683,6.886387,9.885622,4.549633,4.522733,8.201431,9.288235,6.041309,5.125416,6.800230,7.126053,8.412588,9.083983
